# **Zero shot classification: Demonstrated with the case study "Recipes"**

### All required libraries are imported first

In [ ]:
import gradio as gr
from transformers import pipeline
import pandas as pd
import altair as alt
import random

### The "zero-shot-classification" pipeline of transformers is then defined with a variable

In [ ]:
# Load the pipeline and the dataset
zer = pipeline("zero-shot-classification")
URL = "https://raw.githubusercontent.com/laurenzbrahner/BigDataScenarios/main/data/Recipes_Training.csv"
df = pd.read_csv(URL, sep=";")
df_recipe_list = df["ingredients"].to_list()

### Now we define the function that will later process our inputs and deliver the desired result

In [ ]:
# Function to get a random recipe
def get_random_recipe():
    recipe = random.choice(df_recipe_list)
    return ', '.join(eval(recipe))

# Main function for analysis
def analyze_text(text=None, dropdown=[], random_recipe=False,  labels=None):
    # If no text is given, get a random recipe  
    if random_recipe:
        text = get_random_recipe()
    
    # If no labels are given, use the default labels
    if not labels:  
        labels = ['Chinese', 'Indian', 'Italian', 
                  'Mexican', 'Southern Usa', 
                  'Cajun Creole', 'French', 'Thai']
    # If no text is given, use the dropdown
    if not text and dropdown:
      data = zer(dropdown, labels)
      data = pd.DataFrame(data)
      data['scores'] = data['scores'].round(2)
      data = data.rename(columns={"sequence": "Recipe", "scores": "Score", 
                                  "labels": "Cuisine"})
      data_2 = data.drop(columns=["Recipe"], axis=1)
    
    # If no dropdown is given, use the text
    elif text and not dropdown:
      data = zer(text, labels)
      data = pd.DataFrame(data)
      data['scores'] = data['scores'].round(2)
      data = data.rename(columns={"sequence": "Recipe", "scores": "Score", 
                                  "labels": "Cuisine"})
      data_2 = data.drop(columns=["Recipe"], axis=1)
    # If both are given, use the text
    else:
      data = zer(text, labels)
      data = pd.DataFrame(data)
      data['scores'] = data['scores'].round(2)
      data = data.rename(columns={"sequence": "Recipe", "scores": "Score", 
                                  "labels": "Cuisine"})
      data_2 = data.drop(columns=["Recipe"], axis=1)


    # Create the barplot
    color_scale = alt.Scale(range=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f'])

    plot = alt.Chart(data_2).mark_bar(
        opacity=0.8,
        size=30
    ).encode(
        x=alt.X('Cuisine', title='Cuisine', sort=alt.EncodingSortField(field='Score',order='descending'), axis=alt.Axis(labelAngle=0, titleAnchor='start')),
        y=alt.Y('Score', title='Score', axis=alt.Axis(titleAnchor='start')),
        color=alt.Color('Cuisine', scale=color_scale),
        tooltip=[alt.Tooltip('Score',title='Matchscore of the Cuisine')]
    ).properties(
        width=600,
        height=400,
        title='Matchscore of the Cuisines'
    ).configure_axis(
        grid=False
    ).configure_view(
        strokeWidth=0
    )

    plot.save('Barplot.html') 

    # Return the text, the data, the plot and the html file
    if text and not dropdown:
      return text, data_2, plot, 'Barplot.html'
    # If dropdown is given, use the dropdown
    elif dropdown and not text:
      return ', '.join(eval(dropdown)), data_2, plot, 'Barplot.html'
    # If both are given, use the text
    else:
      return text, data_2, plot, 'Barplot.html'

## **Definition of the Gradio UI**

### The checkboxes that can be used to define the labels for the ZSC (if no option is selected, all are selected automatically).

In [ ]:
# Checkbox for random recipe
random_recipe_checkbox = gr.Checkbox(label="Use Random Recipe")

# Checkbox for selecting cuisines
checkbox = gr.CheckboxGroup(['Chinese','Indian','Italian',
                             'Mexican','Southern Usa', 'Cajun Creole', 
                             'French', 'Thai'], label="Choose countries", info="If nothing is selected, all options will be selected automatically.")

dropdown = gr.Dropdown(df_recipe_list, label="Choose a recipe from the dataset")

### The Gradio interface is now defined together with the function, inputs, outputs and title and finally executed.

In [ ]:
# Gradio Interface
demo = gr.Interface(
    fn=analyze_text,
    inputs=[ gr.Textbox(placeholder="Enter ingredients manually", label="Ingredients"), dropdown ,random_recipe_checkbox, checkbox],  # Reihenfolge der Argumente hier beibehalten
    outputs=[gr.Textbox(label="Used Recipe", placeholder="This is the recipe you used"),gr.DataFrame(label="Table sorted score"),
             gr.Plot(label="Barplot"),gr.File(label="Download plot here")],
    title="Zero-Shot Recipe Classification",
    allow_flagging="never"
)

demo.launch()

## **____________________________________________________________________________**

## **The entire code in one piece:**

In [ ]:
import gradio as gr
from transformers import pipeline
import pandas as pd
import altair as alt
import random

# Load the pipeline and the dataset
zer = pipeline("zero-shot-classification")
URL = "https://raw.githubusercontent.com/laurenzbrahner/BigDataScenarios/main/data/Recipes_Training.csv"
df = pd.read_csv(URL, sep=";")
df_recipe_list = df["ingredients"].to_list()

# Function to get a random recipe
def get_random_recipe():
    recipe = random.choice(df_recipe_list)
    return ', '.join(eval(recipe))

# Main function for analysis
def analyze_text(text=None, dropdown=[], random_recipe=False,  labels=None):  
    if random_recipe:
        text = get_random_recipe()

    if not labels:  
        labels = ['Chinese', 'Indian', 'Italian', 
                  'Mexican', 'Southern Usa', 
                  'Cajun Creole', 'French', 'Thai']
    if not text and dropdown:
      data = zer(dropdown, labels)
      data = pd.DataFrame(data)
      data['scores'] = data['scores'].round(2)
      data = data.rename(columns={"sequence": "Recipe", "scores": "Score", 
                                  "labels": "Cuisine"})
      data_2 = data.drop(columns=["Recipe"], axis=1)
    
    elif text and not dropdown:
      data = zer(text, labels)
      data = pd.DataFrame(data)
      data['scores'] = data['scores'].round(2)
      data = data.rename(columns={"sequence": "Recipe", "scores": "Score", 
                                  "labels": "Cuisine"})
      data_2 = data.drop(columns=["Recipe"], axis=1)
    
    else:
      data = zer(text, labels)
      data = pd.DataFrame(data)
      data['scores'] = data['scores'].round(2)
      data = data.rename(columns={"sequence": "Recipe", "scores": "Score", 
                                  "labels": "Cuisine"})
      data_2 = data.drop(columns=["Recipe"], axis=1)



    color_scale = alt.Scale(range=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f'])

    plot = alt.Chart(data_2).mark_bar(
        opacity=0.8,
        size=30
    ).encode(
        x=alt.X('Cuisine', title='Cuisine', sort=alt.EncodingSortField(field='Score',order='descending'), axis=alt.Axis(labelAngle=0, titleAnchor='start')),
        y=alt.Y('Score', title='Score', axis=alt.Axis(titleAnchor='start')),
        color=alt.Color('Cuisine', scale=color_scale),
        tooltip=[alt.Tooltip('Score',title='Matchscore of the Cuisine')]
    ).properties(
        width=600,
        height=400,
        title='Matchscore of the Cuisines'
    ).configure_axis(
        grid=False
    ).configure_view(
        strokeWidth=0
    )

    plot.save('Barplot.html') 

    #return text, data_2, plot, 'Barplot.html'
    if text and not dropdown:
      return text, data_2, plot, 'Barplot.html'
    elif dropdown and not text:
      return ', '.join(eval(dropdown)), data_2, plot, 'Barplot.html'
    else:
      return text, data_2, plot, 'Barplot.html'

# Checkbox for random recipe
random_recipe_checkbox = gr.Checkbox(label="Use Random Recipe")

# Checkbox for selecting cuisines
checkbox = gr.CheckboxGroup(['Chinese','Indian','Italian',
                             'Mexican','Southern Usa', 'Cajun Creole', 
                             'French', 'Thai'], label="Choose countries", info="If nothing is selected, all options will be selected automatically.")

dropdown = gr.Dropdown(df_recipe_list, label="Choose a recipe from the dataset")

# Gradio Interface
demo = gr.Interface(
    fn=analyze_text,
    inputs=[ gr.Textbox(placeholder="Enter ingredients manually", label="Ingredients"), dropdown ,random_recipe_checkbox, checkbox],  # Reihenfolge der Argumente hier beibehalten
    outputs=[gr.Textbox(label="Used Recipe", placeholder="This is the recipe you used"),gr.DataFrame(label="Table sorted score"),
             gr.Plot(label="Barplot"),gr.File(label="Download plot here")],
    title="Zero-Shot Recipe Classification",
    allow_flagging="never"
)

demo.launch()
